# **[MC906] Projeto Final**: Detecção de Desastres

O objetivo desse projeto é construir e avaliar modelos de aprendizado de máquina que classifiquem quais Tweets são sobre desastres reais e quais não são.

## **Acessar Diretório do Projeto**

Esse Notebook assume que você está executando o código dentro da pasta `Projeto Final/Código`, que contém todos os códigos fontes relativos a esse trabalho. Para acessar o diretório no Colab é preciso criar um atalho em seu Drive (right click no diretório -> "Adicionar atalho ao Google Drive") e executar as células abaixo:

In [ ]:
# Conectar ao Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Diretório do Projeto (/content/drive/My Drive/{path até Projeto Final/Código}), 
# dependendo da onde se localiza o atalho no seu Drive
% cd '/content/drive/My Drive/[MC906] Introdução à Inteligência Artificial/Projeto Final/Código'
! ls

/content/drive/.shortcut-targets-by-id/1HmHC25ZqX3hUlCsRT-S0qiSsjwf10jLn/[MC906] Introdução à Inteligência Artificial/Projeto Final/Código
'Attention CNN'			 Modelos
 BERT				 __pycache__
'Convolutional Neural Network'	'Quasi-Recurrent Networks'
'Dense Neural Networks'		'Recurrent Neural Networks'
'Logistic Regression'		 tokenization.py
'Melhor Pré-Processamento'	 utils.py


## **Dependências:**

In [ ]:
# Imports de pacotes instalados
from os.path import join
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd

# Imports locais
from utils import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## **Dataset:**

Utilizamos um *dataset* disponível no site [Kaggle](https://www.kaggle.com/c/nlp-getting-started/data) (em inglês). Cada tweet apresenta três atributos: seu conteúdo (`text`), uma palavra-chave (`keyword`, opcional) e a localização da onde foi enviado (`location`, opcional). Como só usaremos o texto, removemos os dois últimos.

In [ ]:
# Ler e limpar dados (removendo colunas do id, keyword e location)
train = pd.read_csv("../Dataset/train.csv")
train = train.drop(['id','keyword','location'], axis=1)

# Imprimir alguns dados
print(train.head())
vals = train.groupby('target').count()
print("\nSome General insights:")
print(f"Figure of Speech: {vals.iloc[0]['text']*100/len(train):.2f}%")
print(f"Actual Accidents: {vals.iloc[1]['text']*100/len(train):.2f}%")


Some General insights:
Figure of Speech: 57.03%
Actual Accidents: 42.97%


## **Pré-processamento:**

Aplicamos uma série de passos para o pré-processamento textual do dataset:
* Remoção de pontuações, *stop words* e palavras menos frequentes.
* Tokenization: Separação das frases em tokens/palavras e pontuações.
* Stemming: corte de prefixos e sufixo das palavras de forma indiscriminada. Associa as palavras à uma raiz comum.
* One Hot Encoding: Conversão dos valores categóricos em vetores binários.

In [ ]:
# Limpar texto
print(f"Raw Tweet:\n\t",train.text[1])
train.text = train.text.apply(clean_up)
print("\nRemoved Punctuation and Special Chars:\n\t", train.text[1])

# Aplicar tokenização
train.text = train.text.apply(word_tokenize)
print("\nTokenized Tweet:\n\t", train.text[1])

# Remover stop words e aplicar stemming
show_wordcount(train.text, "\nBefore Removal:")
train.text = train.text.apply(stop_words)
print("\nRemoved Stop Words:\n\t", train.text[1])
train.text = train.text.apply(stemming)
print("\nStemmed Tweet:\n\t",train.text[1])
show_wordcount(train.text, "\nAfter Removal:")

Raw Tweet:
	 Forest fire near La Ronge Sask. Canada

Removed Punctuation and Special Chars:
	 Forest fire near La Ronge Sask Canada

Tokenized Tweet:
	 ['Forest', 'fire', 'near', 'La', 'Ronge', 'Sask', 'Canada']

Before Removal:
Amount of Words: 105647
Amount of Distinct Words: 21526

Removed Stop Words:
	 ['fire', 'near', 'Canada', 'Ronge', 'Sask', 'Forest', 'La']

Stemmed Tweet:
	 ['fire', 'near', 'canada', 'rong', 'sask', 'forest', 'la']

After Removal:
Amount of Words: 77292
Amount of Distinct Words: 18502


In [ ]:
show_wordcount(train.text, '\nBefore Removal:')
words, counts = count_words(train.text)

# Criar lista de palavras pouco frequentes
MIN_FREQ = 2
discard = []
for (w,c) in zip(words, counts[0]):
    if c < MIN_FREQ : 
      discard.append(w)

# Criar conjunto com as palavras selecionadas pelo countvectorizer
# descartando as palavras pouco frequentes, e remover as palavras
# do tweet que não pertençam a esse conjunto
print(f"\nDiscarding {len(discard)} Words...")
select = set(words)-set(discard)
train.text = train.text.apply(lambda x : list(set(x) & select))
show_wordcount(train.text, '\nAfter Removal:')


Before Removal:
Amount of Words: 77292
Amount of Distinct Words: 18502

Discarding 13145 Words...

After Removal:
Amount of Words: 62736
Amount of Distinct Words: 5320


In [ ]:
# Aplicar One Hot Encoding 
labels = LabelEncoder().fit(list(select))
encoded = train.text.apply(labels.transform) # Transforming words to labels
encoded = encoded.apply(lambda x : onehotencoding(x, len(select)))

In [ ]:
# Array numpy dos tweets pré-processados e targets
X = np.array(encoded.tolist())
Y = np.array([[x] for x in train.target.tolist()]) 

In [ ]:
# Dividir dataset
X, X_v, Y, Y_v = train_test_split(X, Y, test_size=0.1)

## **Modelo**: Regressão Logística

Para um resultado inicial de referência, foi testado o método mais simples de classificação estudado: a regressão logística. Utilizamos a classe "LogisticRegression", da biblioteca *Scikit-Learn*.

In [ ]:
log = LogisticRegression(max_iter=1000)
log_history = log.fit(X, Y)
print("Training Acc: ", log.score(X,Y))
print("Validation Acc: ", log.score(X_v,Y_v))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Acc:  0.9186980002919282
Validation Acc:  0.7979002624671916
